In [38]:
import json
import re
import numpy as np
import pandas as pd
import ast
import random
from spacy.training import offsets_to_biluo_tags, biluo_to_iob
import spacy
from spacy.training.example import Example
from spacy.lang.ru import Russian
from pymystem3 import Mystem

In [2]:
spacy.require_gpu()

True

In [3]:
with open('dataset\\train.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    data = json.load(f) #загнали все, что получилось в переменную
data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [42]:
nlp = spacy.blank("ru")
max_lengh_tag = 0
count = 0

df = pd.DataFrame([])



for dict_data in data:

    doc = nlp(dict_data['text'])
    #for token in doc:
    #print([t.text for t in doc])
    #print('Длинна токена: ',len(doc))

    label = dict_data.get('label')

    answer_start = dict_data.get('extracted_part').get('answer_start')[0]
    answer_end = dict_data.get('extracted_part').get('answer_end')[0]
    entities = [(answer_start, answer_end, label)]
    tags = offsets_to_biluo_tags(doc, entities)
    iob_tags = biluo_to_iob(tags)

    #print(iob_tags)
    if len(doc) > 512:
        count+=1
    if max_lengh_tag < len(doc):
        max_lengh_tag = len(doc)
    #print(len(tags))
    #print(tags)
print(max_lengh_tag)
print(count)

['Извещение', 'о', 'проведении', 'открытого', 'конкурса', 'в', 'электронной', 'форме', 'для', 'закупки', '№', '0328300032822000806', 'Общая', 'информация', 'Номер', 'извещения', '0328300032822000806', 'Наименование', 'объекта', 'закупки', 'Поставка', 'продуктов', 'питания', 'Способ', 'определения', 'поставщика', '(', 'подрядчика', ',', 'исполнителя', ')', 'Открытый', 'конкурс', 'в', 'бль', 'Порядок', 'внесения', 'денежных', 'средств', 'в', 'качестве', 'обеспечения', 'заявки', 'на', 'участие', 'в', 'закупке', ',', 'а', 'также', 'условия', 'гарантии', 'Обеспечение', 'заявки', 'на', 'участие', 'в', 'закупке', 'может', 'предоставляться', 'участником', 'закупки', 'в', 'виде', 'денежных', 'средств', 'или', 'независимой', 'гарантии', ',', 'предусмотренной', 'ст.', '45', 'Федерального', 'закона', '№', '44-ФЗ', '.', 'Выбор', 'способа', 'обеспечения', 'осуществляется', 'участником', 'закупки', 'самостоятельно', '.', 'Срок', 'действия', 'независимой', 'гарантии', 'должен', 'составлять', 'не', 'ме

D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "РАЗДЕЛ 4. тельно); в) 1 процент цены контракта, ес..." with entities "[(1010, 1179, 'обеспечение исполнения контракта')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


['КОНТРАКТ', '№', 'г.', 'Евпатория', '"', '_', '_', '_', '_', '_', '"', '_', '_', '_', '_', '_', '_', '_', '_', '2022', 'г.', ' ', 'б', ')', 'в', 'случае', ',', 'если', 'цена', 'контракта', 'превышает', 'начальную', '(', 'максимальную', ')', 'цену', 'контракта', ':', '10', 'процентов', 'цены', 'контракта', ',', 'если', 'цена', 'контракта', 'не', 'превышает', '3', 'млн', '.', 'рублей', ';', '5', 'процентов', 'начальной', '(', 'максимальной', ')', 'цены', 'контракта', 'в', 'случае', ',', 'если', 'начальная', '(', 'максимальная', ')', 'цена', 'контракта', 'составляет', 'от', '3', 'млн', '.', 'рублей', 'до', '50', 'млн', '.', 'рублей', '(', 'включительно', ')', ';', '8.12', '.', '3а', 'каждый', 'факт', 'неисполнения', 'или', 'ненадлежащего', 'исполнения', 'Поставщиком', 'обязательства', ',', 'предусмотренного', 'Контрактом', ',', 'которое', 'не', 'имеет', 'стоимостного', 'выражения', ',', 'Поставщик', 'выплачивает', 'Заказчику', 'штраф', '.', 'Размер', 'штрафа', 'определяется', 'в', 'соотв

D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "PAGE ПРОЕКТ КОНТРАКТА КОНТРАКТ № ______ на поставк..." with entities "[(1226, 1355, 'обеспечение исполнения контракта')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ПРОЕКТ Государственный контракт №________ на поста..." with entities "[(1193, 1314, 'обеспечение исполнения контракта')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "УТВ

['Извещение', 'о', 'проведении', 'электронного', 'аукциона', 'для', 'закупки', '№', '0352300000422000101', 'Общая', 'информация', 'Номер', 'извещения', '0352300000422000101', 'Наименование', 'объекта', 'закупки', 'Поставка', 'расходных', 'материалов', '(', 'Реестровый', 'номер', '2022.062392', ')', 'Способ', 'определения', 'поставщика', '(', 'подрядчика', ',', 'исполнителя', ')', 'Электронный', 'аукцион', 'Наименование', ' ', '15', '%', 'Организациям', 'инвалидов', 'в', 'соответствии', 'со', 'ст.', '29', 'Закона', '№', '44-ФЗ', '-', '15', '%', 'Требования', 'к', 'участникам', '1', 'Единые', 'требования', 'к', 'участникам', 'закупок', 'в', 'соответствии', 'с', 'ч', '.', '1', 'ст.', '31', 'Закона', '№', '44-ФЗ', '2', 'Требования', 'к', 'участникам', 'закупок', 'в', 'соответствии', 'с', 'ч', '.', '1.1', 'ст.', '31', 'Закона', '№', '44-ФЗ', 'Ограничения', '1', 'Запрет', 'на', 'допуск', 'товаров', ',', 'работ', ',', 'услуг', 'при', 'осуществлении', 'закупок', ',', 'а', 'также', 'ограничения

D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Приложение № 4 к извещению Проект МУНИЦИПАЛЬНЫЙ КО..." with entities "[(1290, 1409, 'обеспечение исполнения контракта')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "СОГЛАСОВАНО УТВЕРЖДЕНО И.о.начальника МКУ Директор..." with entities "[(1196, 1308, 'обеспечение исполнения контракта')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "1 3

In [22]:
s = '1234'
s[:-1]

'123'

In [23]:
m = Mystem()
data_train = []
count_all = 0
count_space = 0

for dict in data:
    text = dict.get('text')
    label = dict.get('label')
    answer_start = dict.get('extracted_part').get('answer_start')[0]
    answer_end = dict.get('extracted_part').get('answer_end')[0]

    if answer_start == 0 and answer_end == 0:
        row = (text, {'entities': []})
    else:
        count_all+=1
        text_ent = text[answer_start:answer_end+1]
        if text_ent[-1] in [' ', '.', ',', '!', '?', ';',':', '/', '-']:
            count_space+=1
            full_txt = "".join(m.lemmatize(text))
            extracted_part = "".join(m.lemmatize(text_ent[:-1]))
            print()

        '''
        else:
            print('Заканчивается на: (', text_ent[-1], end=') ')
            text_ent = text[answer_start:answer_end+10]
            print('10 символов после ',text_ent[len(text_ent)-10:len(text_ent)])
        '''

print('Кол-во не пустых текстов: ',count_all)
print('Кол-во текстов где следующий символ равен какому то разделителю: ',count_space)
print(count_all - count_space)

None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

### Вывод
Таким образом можем исключить из обучающей выборки срезы, следующий символ которых не заканчивается на разделитель. Удаляем 1492 - 1447 =  45 значение. Эти значения можно считать выбросами

In [56]:
data_train = []
for dict in data:
    text = dict.get('text')
    label = dict.get('label')
    answer_start = dict.get('extracted_part').get('answer_start')[0]
    answer_end = dict.get('extracted_part').get('answer_end')[0]

    if answer_start == 0 and answer_end == 0:
        row = (text, {'entities': []})

    else:
        label_split = label.split(' ')

        first_word = '[Оо]беспечен.{1,100}'
        sec_word = '[' +label_split[1][0].upper()+label_split[1][0].lower()+']'+ label_split[1][1:-2] +'.{1,100}.'

        if label_split[2] == 'контракта':
            third_word = '([Кк]онтракт.{1,100} | [Дд]оговор.{1,100})'
        else:
            third_word = '[Оо]бязательст.{1,100}'
        pattern = first_word + sec_word + third_word
        res = re.search(pattern, text)

        if res:
            pass
        else:
            print(pattern)
            print(res)
            #print(text)
            print(text[answer_start:answer_end])

        row = (text, {'entities': [(answer_start, answer_end, label)]})
    data_train.append(row)


[Оо]беспечен.{1,100}[Ии]сполнен.{1,100}.([Кк]онтракт.{1,100} | [Дд]оговор.{1,100})
None
Размер обеспечения договора 5 % от цены договора
[Оо]беспечен.{1,100}[Ии]сполнен.{1,100}.([Кк]онтракт.{1,100} | [Дд]оговор.{1,100})
None
при отсутствии в конкретной закупочной документации требования об обеспечении договора: ОДi = (30 – Zi) %
[Оо]беспечен.{1,100}[Ии]сполнен.{1,100}.([Кк]онтракт.{1,100} | [Дд]оговор.{1,100})
None
при отсутствии в конкретной закупочной документации требования об обеспечении договора: ОДi = (30 – Zi) %
[Оо]беспечен.{1,100}[Гг]арантийн.{1,100}.[Оо]бязательст.{1,100}
None
Размер обеспечения исполнения обязательств Участника в гарантийный период составляет не менее 5 (пяти) процентов от цены сделки - для сделок с ценой не более 100 млн рублей


### Вывод

Один из вариантов это фильтрация через регулярные выражения

In [50]:
s='Размер обеспечения исполнения обязательств Участника в гарантийный период составляет не менее 5 (пяти) процентов от цены сделки - для сделок с ценой не более 100 млн рублей'
res = re.search('[Оо]беспечен.{1,100}[Гг]арантийн.{1,10}.[Оо]бязательс.{1,10}', s)
print(res)

None


### Загружаем данные формата json и переводим в DataFrame

In [54]:
data = pd.read_json('dataset\\train.json')
data.head()

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1799 non-null   int64 
 1   text            1799 non-null   object
 2   label           1799 non-null   object
 3   extracted_part  1799 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


### Создадим отдельный датафрейм с таргетом
Сам тарегет удаляем из обучающей выбоорки

In [56]:
target = pd.DataFrame(data['extracted_part'].to_list())
target['id'] =  data['id']
data = data.drop(columns=['extracted_part'])
target

,text,answer_start,answer_end,id
0,[Размер обеспечения исполнения контракта 6593....,[1279],[1343],809436509
1,[Поставщик должен предоставить обеспечение исп...,[1222],[1318],854885310
2,[Размер обеспечения исполнения контракта 10.00%],[1297],[1343],4382157
3,[Размер обеспечения исполнения контракта 10.00%],[1304],[1350],184555082
4,[Размер обеспечения исполнения контракта 10.00%],[1302],[1348],211645258
...,...,...,...,...
1794,[Заказчиком установлено требование обеспечения...,[1213],[1402],938335285
1795,[],[0],[0],214609521
1796,[],[0],[0],94811748
1797,[],[0],[0],903507299


In [57]:
def func(row):
    if row.to_list()[0]:
        return row.to_list()[0][0]
    else:
        return np.nan

target['text'] = target[['text']].apply(lambda row : func(row), axis=1)
target['answer_start'] = target[['answer_start']].apply(lambda row : func(row), axis=1)
target['answer_end'] = target[['answer_end']].apply(lambda row : func(row), axis=1)
print(target.head())
print(target.info())

                                                text  answer_start  \
0  Размер обеспечения исполнения контракта 6593.2...          1279   
1  Поставщик должен предоставить обеспечение испо...          1222   
2     Размер обеспечения исполнения контракта 10.00%          1297   
3     Размер обеспечения исполнения контракта 10.00%          1304   
4     Размер обеспечения исполнения контракта 10.00%          1302   

   answer_end         id  
0        1343  809436509  
1        1318  854885310  
2        1343    4382157  
3        1350  184555082  
4        1348  211645258  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          1799 non-null   object
 1   answer_start  1799 non-null   int64 
 2   answer_end    1799 non-null   int64 
 3   id            1799 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 56.3+ KB
None


In [58]:
target['text']

0       Размер обеспечения исполнения контракта 6593.2...
1       Поставщик должен предоставить обеспечение испо...
2          Размер обеспечения исполнения контракта 10.00%
3          Размер обеспечения исполнения контракта 10.00%
4          Размер обеспечения исполнения контракта 10.00%
                              ...                        
1794    Заказчиком установлено требование обеспечения ...
1795                                                     
1796                                                     
1797                                                     
1798                                                     
Name: text, Length: 1799, dtype: object

In [59]:
#

### Возможные решения
1) Построить модель на основе регулярных выражений
2) Построить модель на основе spaCy и textacy
3) Неронная сеть

### Тестирование теорий

In [49]:
print(data.iloc[0]['text'][target.iloc[0]['answer_start']: target.iloc[0]['answer_end']])

Размер обеспечения исполнения контракта 6593.25 Российский рубль


In [50]:
target.iloc[0]

text            Размер обеспечения исполнения контракта 6593.2...
answer_start                                                 1279
answer_end                                                   1343
id                                                      809436509
Name: 0, dtype: object

In [51]:
for i in range(10):
    print('Срез: ',data.iloc[i]['text'][target.iloc[i]['answer_start']: target.iloc[i]['answer_end']])
    print('Должно быть: ',target.iloc[i]['text'],'\n')

Срез:  сполнения контракта Размер обеспечения исполнения контракта 6593.25 Российский рубль Порядок обеспечения
Должно быть:  Размер обеспечения исполнения контракта 6593.25 Российский рубль 

Срез:  102810945370000005. Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта. В случае если обесп
Должно быть:  Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта. 

Срез:  сполнения контракта Размер обеспечения исполнения контракта 10.00% Порядок обеспечения
Должно быть:  Размер обеспечения исполнения контракта 10.00% 

Срез:  сполнения контракта Размер обеспечения исполнения контракта 10.00% Порядок обеспечения
Должно быть:  Размер обеспечения исполнения контракта 10.00% 

Срез:  сполнения контракта Размер обеспечения исполнения контракта 10.00% Порядок обеспечения
Должно быть:  Размер обеспечения исполнения контракта 10.00% 

Срез:  ции о закупке. 7.4. Размер обеспечения исполнения контракта устанавливае